In [12]:
import os

folder_path = '/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/text'

text_file = len([
    f for f in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, f)) and not f.startswith('.')
])

print("file count", text_file)


file count 40


In [ ]:
# Create chunk
# Chunk and cleaned text

from langchain.text_splitter import RecursiveCharacterTextSplitter

raw_text = ''
# Load cleaned text 
for i in range(text_file):
    with open(f'/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/text/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ_page_{i+1}.text', "r", encoding="utf-8") as f:
        raw_text += f.read() + '/n/n'
        

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # number of characters per chunk
    chunk_overlap=50,   # how much to overlap between chunks
    separators=["\n\n", "\n", ".", "។", "៕", " "],  # include Khmer punctuation
)

# Create chunks
chunks = text_splitter.split_text(raw_text)


# Create chunk folder
folder_dir =  '/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ'
chunk_folder = os.path.join(folder_dir, 'chunk_text')
os.makedirs(chunk_folder, exist_ok = True)

# Optional: Save all chunks to a single file
with open("/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/chunk_text/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ_chunks.txt", "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i + 1} ---\n{chunk}\n\n")


In [ ]:
# embed chunk text

In [ ]:
#pip install sentence-transformers

In [1]:
def load_chunks_from_file(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # Split on "--- Chunk"
    raw_chunks = text.split('--- Chunk')[1:]  # skip the header before first chunk
    chunks = ["--- Chunk" + chunk.strip() for chunk in raw_chunks if chunk.strip()]
    return chunks
    
chunks = load_chunks_from_file('/Users/cheamenghour/Documents/Cambodia Agriculture/All_chunks/all_chunks.txt') # Load all chunk text


In [2]:
# Check chunk lens

print(len(chunks))

8


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
embeddings_model = SentenceTransformer('distiluse-base-multilingual-cased')
embeddings = embeddings_model.encode(chunks, convert_to_tensor=True)
vectors = np.array(embeddings).astype("float32")

In [ ]:
# Save database for further use

import faiss
import numpy as np

dimension = vectors.shape[1]  
index = faiss.IndexFlatL2(dimension)
index.add(vectors)


# Save index for later use
faiss.write_index(index, '/Users/cheamenghour/Documents/Cambodia Agriculture/Vector database/khmer_agri_index.faiss') # Dir to save database


In [ ]:
print("Chunks:", len(chunks), "| FAISS vectors:", index.ntotal)